In [ ]:
df = pd.read_csv("data/bank-full.csv", sep= ";")

In [ ]:
from sklearn.model_selection import train_test_split
# Define las variables independientes y la variable dependiente
features = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaing', 'pdays', 'previous', 'poutcome' ]
X = df[features]
y = df['y']

# Dividir el conjunto de datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
import statsmodels.api as sm
# agregar constante explíticamente
X_train = sm.add_constant(X_train)
# regresión usando mínimos cuadrados ordinarios (ordinary least squares - OLS) 
linreg1 = sm.OLS(y_train, X_train).fit()
# resumen de resultados
print(linreg1.summary())

In [ ]:
y_pred = linreg1.predict(sm.add_constant(X_test))

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_error
import numpy as np
# Evaluar el modelo con R^2 y RMSE, MAE, MSE
r2 = r2_score(y_test, y_pred)
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
MAE = mean_absolute_error(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
print(f"R²: {r2}")
print(f"MAE: {MAE}")
print(f"MSE: {MSE}")
print(f"RMSE: {RMSE}")

## Redes

In [ ]:
train = df.sample(frac=0.8, random_state=100)
train.head()

In [ ]:
test = df.drop(train.index)
test.head()

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.corr()

In [ ]:
sns.heatmap(train.corr())

Seleccionamos ahora las características y las etiquetas (MPG)

In [ ]:
train_X = train.copy()
test_X = test.copy()

In [ ]:
train_y = train_X.pop('MPG')
test_y = test_X.pop('MPG')

Construyamos los modelos usando tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
tf.keras.layers.Normalization

In [ ]:
linear_model = tf.keras.Sequential([
    norm,
    tf.keras.layers.Dense(units=1)
])
linear_model.summary()

In [ ]:
linear_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [ ]:
%%time
history = linear_model.fit(
    train_X,
    train_y,
    epochs=100,
    validation_split = 0.2)

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylim([0, 10])
plt.xlabel('Epoch')
plt.ylabel('Error [MPG]')
plt.legend()

In [ ]:
test_res['linear_model'] = linear_model.evaluate(
    test_X, test_y, verbose=0)
test_res

In [ ]:
dnn_model = tf.keras.Sequential([
      norm,
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(1)
  ])

In [ ]:
dnn_model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))

In [ ]:
dnn_model.summary()

In [ ]:
%%time
history = dnn_model.fit(
  train_X,
  train_y,
  validation_split=0.2,
  epochs=100)

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylim([0, 10])
plt.xlabel('Epoch')
plt.ylabel('Error [MPG]')
plt.legend()

In [ ]:
test_res['dnn_model'] = dnn_model.evaluate(test_X, test_y, verbose=0)

In [ ]:
test_res

In [ ]:
test_pred = dnn_model.predict(test_X).flatten()

plt.axes(aspect='equal')
plt.scatter(test_y, test_pred)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_=plt.plot(lims, lims)